In [ ]:
pip install pyspark

     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 51.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805911 sha256=160075f4e3f4cb6ae90939805f1092c53701c11986f1d8e44973582263e6a323
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


# Создаём точку входа в Spark

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


spark = SparkSession.builder\
        .master('local[4]')\
        .appName('Lesson_2')\
        .config('spark.ui.port', '4050')\
        .config('spark.executor.instances', 2)\
        .config('spark.executor.memory', '5g')\
        .config('spark.executor.cores', 2)\
        .getOrCreate()

sc = spark.sparkContext

In [ ]:
spark

# DataFrame
Теперь мы знаем про Dataframe. Нужно сделать практически всё то же самое, но используя датафрейм.

# Самостоятельная работа

Требуется выяснить:
1. Какое соотношение сторон экрана телефона самое популярное,
2. Плотность пикселей у экрана.

Всего 8 баллов. 


## Считывание данных
Данные взяты отсюда: https://www.kaggle.com/iabhishekofficial/mobile-price-classification
Скачиваем и копируем в папку с нотебуком

Внутри содержится следующая информация:

* id: ID
* battery_power: Total energy a battery can store in one time (mAh)
* blue: Support bluetooth or not
* clock_speed: Speed at which microprocessor executes instructions
* dual_sim: Support dual sim or not
* fc: Front Camera mega pixels
* four_g: Support 4G or not
* int_memory: Internal Memory (GB)
* m_dep: Mobile Depth (cm)
* mobile_wt: Weight of mobile phone
* n_cores: Number of cores of processor
* pc: Primary Camera mega pixels
* px_height: Pixel Resolution Height
* px_width: Pixel Resolution Width
* ram: Random Access Memory (MB)
* sc_h: Screen Height of mobile (cm)
* sc_w: Screen Width of mobile (cm)
* talk_time: Time that a single battery charge will last
* three_g: Support 3G or not
* touch_screen: Has touch screen or not
* wifi: Support wifi or not

# Считывание данных

In [ ]:
# Прочитайте содержимое каждого файла в датафрейм и выведите первые несколько строк
# Применив cast к колонке можно изменить тип данных (F.col().cast(IntegerType()).alias())
# Приведите все данные к правильному типу, либо считайе сразу верно

train = spark.read.csv('train.csv', header=True)
test = spark.read.csv('test.csv', header=True)

In [ ]:
train.show()

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|   1|          1|
|         1021|   1|        0.5|       1|  0|     1|        53|  0.7|      136|      3|  6|      905|    1988|2631|  17|   3|        7|      1|           1|   0|          2|
|          563|   1|        0.5|       1|  2|     1|        41|  0.9|      145|      5|  6|     1263|    1716|2603|  11|   2|     

## Объединим train и test
Найти нужную функцию можно [здесь](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html)

PS: нужно сделать средствами pyspark pd.concat([train, test,], axis=0)

In [ ]:
# Объедините train и test (1 балла)

data = train.drop('price_range').unionByName(test.drop('id'))

In [ ]:
data.count()

3000

## Рассчитайте соотношение сторон телефона и экрана

In [ ]:
# Выведите отсортированное распределение плотности пикселей (1 балла)
# в пикселях на дюйм

data = data\
    .withColumn('ppi',
        (F.pow(F.pow(F.col('px_height'), F.lit(2)) + F.pow(F.col('px_width'), F.lit(2)), F.lit(0.5)) /
        F.pow(F.pow(F.col('sc_h'), F.lit(2)) + F.pow(F.col('sc_w'), F.lit(2)), F.lit(0.5)))
    )
data.show()

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+------------------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|               ppi|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+------------------+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|   1|  66.3287847944442|
|         1021|   1|        0.5|       1|  0|     1|        53|  0.7|      136|      3|  6|      905|    1988|2631|  17|   3|        7|      1|           1|   0|126.53313490223783|
|          563|   1|        0.5|       1|  2|     1|        41|  0.9|      145|      5|  6|    

## Сохранение


In [ ]:
# Сохраните результат в csv sep=';', encoding='cp1251'
# с колонками id, плотность пикселей и общей инфой в формате "2 ядра 2 гига"


data\
.select(
    F.monotonically_increasing_id(), 
    'ppi', 
    F.concat_ws(' ',
        F.col('n_cores'), 
        F.lit('ядра')).alias('desc'))\
.write.option('encoding', 'cp1252').csv('results.csv', mode='overwrite', sep=';')

+-----------------------------+------------------+------+
|monotonically_increasing_id()|               ppi|  desc|
+-----------------------------+------------------+------+
|                            0|  66.3287847944442|2 ядра|
|                            1|126.53313490223783|3 ядра|
|                            2|190.57439492229798|5 ядра|
|                            3|120.78457062058878|6 ядра|
|                            4|207.51357149097785|2 ядра|
|                            5|113.61963162083906|1 ядра|
|                            6| 71.20920637783695|8 ядра|
|                            7| 77.27297850831926|4 ядра|
|                            8| 54.07185513038533|7 ядра|
|                            9|  77.8081655680639|5 ядра|
|                           10|168.70500677319814|5 ядра|
|                           11|61.062375636815446|8 ядра|
|                           12|53.516842854710724|4 ядра|
|                           13| 209.3769805876472|4 ядра|
|             